In [ ]:
# rag_demo.py
#
# 1. Loads a few sample documents
# 2. Embeds them using OpenAI
# 3. Stores in FAISS for similarity search
# 4. Retrieves relevant chunks for a question
# 5. Asks GPT-4 to answer using those chunks
#
# Prereqs:
# pip install openai faiss-cpu python-dotenv
# export OPENAI_API_KEY="sk-..."
# or use a .env file
#
# Note: This example omits error handling for simplicity.
# In production, you should add proper exception handling for API calls.

import os
import faiss
import numpy as np
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Sample docs
documents = [
    "Employees are entitled to 90 days of paid holiday per year, plus bank holidays.",
    "The company operates a flexible working policy, including remote work options.",
    "Employees can use the company blimp once per year.",
    "The IT helpdesk is available from 7am to 7:05am, Mondays only.",
]

# ────────────────────────────────────────────────────────────────────────────────
# Step 1 – Embed documents
# ────────────────────────────────────────────────────────────────────────────────

def get_embedding(text: str) -> List[float]:
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-3-small" # https://platform.openai.com/docs/models/text-embedding-3-small
    )
    return response.data[0].embedding

doc_embeddings = [get_embedding(doc) for doc in documents]

# ────────────────────────────────────────────────────────────────────────────────
# Step 2 – Store the document vectors in an in-memory table, for similarity searches
# ────────────────────────────────────────────────────────────────────────────────

embedding_size = len(doc_embeddings[0])
faiss_index = faiss.IndexFlatL2(embedding_size) # Exact L2 index, fine for <10k docs
faiss_index.add(np.array(doc_embeddings).astype("float32"))  # Add all document vectors to the index

# ────────────────────────────────────────────────────────────────────────────────
# Step 3 – Ask a question using RAG
# ────────────────────────────────────────────────────────────────────────────────
def retrieve_context(question: str, top_k: int = 2) -> List[str]:
    """Finds the top_k most relevant documents to the question using vector similarity."""
    question_vector = np.array(get_embedding(question)).astype("float32")
    distances, indices = faiss_index.search(np.array([question_vector]), top_k)
    return [documents[i] for i in indices[0]]

def ask_with_context(question: str) -> str:
    context_chunks = retrieve_context(question)
    context = "\n".join(f"- {chunk}" for chunk in context_chunks)

    prompt = f"""
You are a helpful company assistant. Use the context below to answer the user's question.
If the answer is not in the context, say "Sorry, I don't know."

Context:
{context}

Question: {question}
Answer:
""".strip()

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()


In [ ]:
ask_with_context("How many days holiday do I get?")

In [ ]:
ask_with_context("How many days holiday do I have if I have used 20 days already")

In [ ]:
ask_with_context("Can I bring my hamster to work?")

In [ ]:
ask_with_context("When is the IT helpdesk available?")

In [ ]:
ask_with_context("How long is the IT helpdesk open for?")

In [ ]:
ask_with_context("Can I use the company blimp?")

In [ ]:
ask_with_context("Can I use the company blimp? I used it yesterday")

In [ ]:
ask_with_context("I have used the company blimp 3 times this week")